In [ ]:
from morphcloud.api import MorphCloudClient
import os

In [ ]:
client = MorphCloudClient()

In [ ]:
# initialize base instance
snapshot = client.snapshots.create(
    vcpus=1,
    memory=4096,
    disk_size=8192
)

In [ ]:
snapshot = (
    snapshot.setup("apt update -y")
    .setup("apt-get install -y python3-pip")
    .setup("curl -LsSf https://astral.sh/uv/install.sh | bash")
)

# TMP: until we find out how to upload files to a snapshot with the setup api we will manually do it to an instance

In [ ]:
base_instance = client.instances.start(snapshot_id=snapshot.id)

In [ ]:
result = base_instance.exec(command="mkdir app")
# upload project to base instance
with base_instance.ssh() as ssh:
    sftp = ssh._client.open_sftp()
    sftp.put("/home/asusevski/morph/chess/config.yaml", "app/config.yaml")
    sftp.put("/home/asusevski/morph/chess/pyproject.toml", "app/pyproject.toml")
    sftp.put("/home/asusevski/morph/chess/requirements.txt", "app/requirements.txt")
    sftp.put("/home/asusevski/morph/chess/schemas.py", "app/schemas.py")
    sftp.put("/home/asusevski/morph/chess/chess_game.py", "app/chess_game.py")
    sftp.put("/home/asusevski/morph/chess/chess_engine.py", "app/chess_engine.py")
    sftp.put("/home/asusevski/morph/chess/agent.py", "app/agent.py")

# Remaining to setup: uv add pyproject.toml
- NOTE: can just keep the tmp instance i guess

In [ ]:
# branch from instance with files transferred + uv + deps installed
base_snapshot, clones = base_instance.branch(count=3)
print(f"Snapshot created: {base_snapshot.id}. This is the base snapshot for all chess branches.")

In [ ]:
[clone.id for clone in clones]

In [ ]:
NUM_MOVES = 5
game_ids = [1,2,3]
strategies = ["aggressive", "defensive", "balanced"]
#instance_startup = f"export OPENAI_API_KEY={os.environ.get('OPENAI_API_KEY')} && cd app && mkdir chess_autosaves && uv add pyproject.toml && source .venv/bin/activate && python agent.py"
run_agent_cmd = f"export OPENAI_API_KEY={os.environ.get('OPENAI_API_KEY')} && cd app && mkdir chess_autosaves && uv add pyproject.toml && uv run agent.py --game-id {game_id} --moves {NUM_MOVES}"

In [ ]:
for clone, game_id, strategy in zip(clones, game_ids, strategies):
    print(f"Starting game {game_id} on instance {clone.id} with strategy {strategy}")
    run_agent_cmd = f"export OPENAI_API_KEY={os.environ.get('OPENAI_API_KEY')} && cd app && mkdir chess_autosaves && uv add pyproject.toml && uv run agent.py --game-id {game_id} --moves 2"
    
    result = clone.exec(command=run_agent_cmd)
    print("="*50)
    print(f"Stdout:\n{result.stdout}")
    print("-"*50)
    print(f"Stderr:\n{result.stderr}")
    print('-'*50)
    # check that the game state saved
    result = clone.exec(command=f"cat app/chess_autosaves/game_id_{game_id}.json")
    print(f"Stdout:\n{result.stdout}")
    print("-"*50)
    print(f"Stderr:\n{result.stderr}")
    print("="*50)

In [ ]:
print(clones[1].exec(f"cat app/chess_autosaves/game_id_2.json").stdout)

In [ ]:
print(clones[0].exec(f"ls app/chess_autosaves").stdout)

In [ ]:
[x.id for x in clones]

# utils

In [ ]:
# delete all instances
instances = client.instances.list()

for instance in instances:
    print(f"Stopping instance ID: {instance.id}, Current Status: {instance.status}, Snapshot ID: {instance.refs.snapshot_id}")
    instance.stop()


In [ ]:
# delete all snapshots
snapshots = client.snapshots.list()

for snapshot in snapshots:
    print(f"Deleting snapshot ID: {snapshot.id}, Created At: {snapshot.created}")
    snapshot.delete()

In [ ]:
# reset autosaves
for clone in clones:
    _ = clone.exec(command="rm -rf app/chess_autosaves")

In [ ]:
# update agent.py
for clone in clones:
    with clone.ssh() as ssh:
        sftp = ssh._client.open_sftp()
        sftp.put("/home/asusevski/morph/chess/agent.py", "app/agent.py")